In [3]:
print ("Hello Dengu u")

Hello Dengu u


In [4]:
"""Naa istam"""

'Naa istam'

In [5]:
5+5

10

In [6]:
from pyspark.sql import *


In [7]:
import pyspark.mllib

In [8]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [14]:
raw_data = sc.textFile("hour_noheader.csv")

In [15]:
raw_data

hour_noheader.csv MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [16]:
records = raw_data.map(lambda x: x.split(","))

In [18]:
first = records.first()

In [19]:
print (first)

['1', '2011-01-01', '1', '0', '1', '0', '0', '6', '0', '1', '0.24', '0.2879', '0.81', '0', '3', '13', '16']


In [81]:
records.cache()

['1',
 '2011-01-01',
 '1',
 '0',
 '1',
 '0',
 '0',
 '6',
 '0',
 '1',
 '0.24',
 '0.2879',
 '0.81',
 '0',
 '3',
 '13',
 '16']

In [21]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()

In [22]:
print ("Mapping of first categorical feasture column: %s" % (get_mapping(records, 2)))

Mapping of first categorical feasture column: {'1': 0, '4': 1, '2': 2, '3': 3}


In [24]:
mappings = [get_mapping(records, i) for i in range(2,10)]
cat_len = sum(map(len, mappings))
num_len = len(records.first()[11:15])
total_len = num_len + cat_len

In [26]:
print ("Feature vector length for categorical features: %d " % cat_len)
print ("Feature vector length for numerical features: %d " % num_len)
print ("Total feature vector length: %d " % total_len)

Feature vector length for categorical features: 57 
Feature vector length for numerical features: 4 
Total feature vector length: 61 


In [82]:
from pyspark.mllib.regression import LabeledPoint

import numpy as np

def extract_features(record):
    cat_vec = np.zeros(cat_len)
    i = 0
    step = 0
    for field in record[2:9]:
        m = mappings[i]
        idx = m[field]
        cat_vec[idx + step] = 1
        i = i + 1
        step = step + len(m)
    num_vec = np.array([float(field) for field in record[10:14]])
    return np.concatenate((cat_vec, num_vec))

def extract_label(record):
    return float(record[-1])

extract_features(records.first())

array([1.    , 0.    , 0.    , 0.    , 1.    , 0.    , 1.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 1.    ,
       0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.24  , 0.2879, 0.81  , 0.    ])

In [78]:
data = records.map(lambda r: LabeledPoint(extract_label(r), extract_features(r)))

In [79]:
first_point = data.first()

In [33]:
print ("Raw Data" + str(first[2:]))

print ("Label:" + str(first_point.label))

print ("Linear Model feature vector:\n" + str(first_point.features))

print ("Linear Model feature vector length: " + str(len(first_point.features)))

Raw Data['1', '0', '1', '0', '0', '6', '0', '1', '0.24', '0.2879', '0.81', '0', '3', '13', '16']
Label:16.0
Linear Model feature vector:
[1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.24,0.2879,0.81,0.0]
Linear Model feature vector length: 61


In [123]:
# Extract features of decision tree. 

def extract_features_dt(record):
    return np.array(list(map(float, record[2:14])))

# def extract_features_dt(record):
#     cat_vec = np.zeros(cat_len)
#     i = 0
#     step = 0
#     for field in record[2:9]:
#         m = mappings[i]
#         idx = m[field]
#         cat_vec[idx + step] = 1
#         i = i + 1
#         step = step + len(m)
#     num_vec = np.array([float(field) for field in record[10:14]])
#     return np.concatenate((cat_vec, num_vec))



data_dt = records.map(lambda r: LabeledPoint(extract_label(r), extract_features_dt(r)))

# label = extract_label(records.first())
# print (label)
# features_array = extract_features_dt(records.first())
# print (features_array)

# LabeledPoint(label, features_array)

data_dt.count()

first_point_dt = data_dt.first()

print ("Decision Tree feature vector: "  + str(first_point_dt.features))

print ("Decision Tree feature vector length:" + str(len(first_point_dt.features)))


Decision Tree feature vector: [1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0]
Decision Tree feature vector length:12


In [40]:
data_dt.__dict__

{'func': <function pyspark.rdd.RDD.map.<locals>.func(_, iterator)>,
 'preservesPartitioning': False,
 '_prev_jrdd': JavaObject id=o57,
 '_prev_jrdd_deserializer': AutoBatchedSerializer(PickleSerializer()),
 'is_cached': False,
 'is_checkpointed': False,
 'ctx': <SparkContext master=local[2] appName=PySparkShell>,
 'prev': PythonRDD[5] at RDD at PythonRDD.scala:52,
 '_jrdd_val': JavaObject id=o777,
 '_id': None,
 '_jrdd_deserializer': AutoBatchedSerializer(PickleSerializer()),
 '_bypass_serializer': False,
 'partitioner': None}

In [46]:
from pyspark.mllib.regression import LinearRegressionWithSGD
linear_model = LinearRegressionWithSGD.train(data, iterations=10, step=0.1, intercept=False)

In [47]:
true_vs_predicted = data.map(lambda p: (p.label, linear_model.predict(p.features)))

In [48]:
print ("Linear Model predictions: " + str(true_vs_predicted.take(5)))

Linear Model predictions: [(16.0, 117.89250386724845), (40.0, 116.22496123192113), (32.0, 116.02369145779235), (13.0, 115.67088016754433), (1.0, 115.56315650834316)]


In [132]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

dt_model = DecisionTree.trainRegressor(data_dt,{})

preds = dt_model.predict(data_dt.map(lambda p: p.features))

actual = data.map(lambda p: p.label)

true_vs_predicted_dt = actual.zip(preds)

print ("Decision Tree predictions: " + str(true_vs_predicted_dt.take(5)))

print ("Decision Tree depth: " + str(dt_model.depth()))

print ("Decision Tree number of nodes: " + str(dt_model.numNodes()))

Decision Tree predictions: [(16.0, 54.913223140495866), (40.0, 54.913223140495866), (32.0, 53.171052631578945), (13.0, 14.284023668639053), (1.0, 14.284023668639053)]
Decision Tree depth: 5
Decision Tree number of nodes: 63


In [63]:
def squared_error(actual, predicted):
    return (predicted - actual)**2

In [64]:
def abs_error(actual, pred):
    return np.abs(pred - actual)

In [66]:
def squared_log_error(pred, actual):
    return (np.log(pred + 1) - np.log(actual + 1))**2

In [137]:
# 6.1 Linear Model

mse = true_vs_predicted.map(lambda args: squared_error(args[0], args[1])).mean()

mae = true_vs_predicted.map(lambda args: abs_error(args[0], args[1])).mean()

rmsle = np.sqrt(true_vs_predicted.map(lambda args: squared_log_error(args[0], args[1])).mean())

print ("Linear Model – Mean Squared Error: %2.4f" % mse)

print ("Linear Model – Mean Absolute Error: %2.4f" % mae)

print ("Linear Model – Root Mean Squared Log Error: %2.4f" % rmsle)

Linear Model – Mean Squared Error: 30679.4539
Linear Model – Mean Absolute Error: 130.6429
Linear Model – Root Mean Squared Log Error: 1.4653


In [138]:
## 6.2 Decision Tree


mse_dt = true_vs_predicted_dt.map(lambda args: squared_error(args[0], args[1])).

mean()

mae_dt = true_vs_predicted_dt.map(lambda (t, p): abs_error(t, p)).

mean()

rmsle_dt = np.sqrt(true_vs_predicted_dt.map(lambda (t, p): squared_

log_error(t, p)).mean())

print “Decision Tree – Mean Squared Error: %2.4f” % mse_dt

print “Decision Tree – Mean Absolute Error: %2.4f” % mae_dt

print “Decision Tree – Root Mean Squared Log Error: %2.4f” %

rmsle_dt